# Part 1. Benchmarking and Profiling

The Julia set is an interesting CPU-bound problem, it is a fractal sequence that generates a complex output image, named after Gaston Julia.

## Step 1

After reading the sections **"Introducing the Julia Set"** and **"Calculating the Full Julia Set"** on **Chapter 2** from the book **_High Performance Python_**, I'll implement the chapter functions in order to calculate the Julia Set and make a representation for the false gray and pure gray scale.

In [1]:
# Example 2-1. Defining global constants for the coordinate space
'''Julia set generator without optional PIL-based image drawing'''
import time

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

To generate the plot, two lists of input data are created. The first is `zs` (complex z-coordinates) and the second is `cs` (a complex initial condition). To build them, it is necessary to know the coordinates for each z. In the following code, those coordinates are build using `xcoord` and `ycoord` and a specified `x_step`. 

In [2]:
# Example 2-2. stablishing the coordinate lists as inputs to our
# calculation function

def calc_pure_python(desired_width, max_iterations, return_output=False):
    '''Create a list of complex coordinates (zc) and complex
    parameters (cs), build Julia set, and display'''
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # Build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed;
    # we use it to simulate a real-world scenario with several inputs to
    # our function.
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))
    
    print("Lenght of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")
    
    # This sum is expected for a 1000^2 grid with 300 iterations.
    # It catches minor errors we might introduce when we're
    # working on a fixed set of inputs.
    assert sum(output) == 33219980
    if return_output:
        return output

Next, the `calculate_z_serial_purepython` function is defined. Also, an `output` list is defined at the start that has the same length as the input `zs` and `cs` lists.

In [3]:
# Example 2-3. Our CPU-bound calculation function

def calculate_z_serial_purepython(maxiter, zs, cs):
    '''Calculate output list using Julia update rule'''
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

Now a calculation routine is called. By wrapping it in a `__main__` check, the module can be safely imported without starting the calculations for some of the profiling methods. Here the method to plot the output is not used.

In [4]:
# Example 2-4. main for our code
if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python(desired_width=1000, max_iterations=300)

Lenght of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 4.338740110397339 seconds


Now, the representation for false gray and pure grayscale will be added. 

First, the module `PIL` will be assed to support graphical output.

In [5]:
%pip install Pillow

from PIL import Image

Note: you may need to restart the kernel to use updated packages.


And the `array` module is used for its efficiency in handling large sequences of numbers. 

In [6]:
import array

**False Grayscale Plot**

The following fuction creates a false grayscale image representation of the Julia set. The false grayscale refers to a grayscale image where the colors are derived from the number of iterations needed to determine if a point escapes to infinity according to the Julia set rules. In a true grayscale image, each pixel's color is a shade of gray, varying from black at the weakest intensity to white at the strongest intensity. However, in false grayscale, the intensity is often translated into a colorized scale (not necessarily shades of gray) to highlight differences between areas with different escape times.

In [7]:
def false_grayscale(desired_width, max_iterations):
    # Calculate the Julia set
    output = calc_pure_python(desired_width, max_iterations, return_output=True)
    
    # Find the maximum iteration value for normalization
    max_value = float(max(output))
    
    # Normalize iteration counts to the 0-255 grayscale range.
    output_raw_limited = [int(float(o) / max_value * 255) for o in output]
    
    # Convert grayscale values to equivalent RGB values.
    # Since the image is grayscale, R, G, and B components are the same.
    # Each value in the generator will be a 32-bit integer where the least significant
    # 24 bits are R, G, and B values (all three are the same for grayscale),
    # multiplied by 16 to fill the 32 bits (since we're ignoring the alpha channel).
    output_rgb = ((o + (256 * o) + (256 ** 2) * o) * 16 for o in output_raw_limited)
    
    # Convert the RGB generator to an array of unsigned integers ('I').
    output_rgb = array.array('I', output_rgb)
    
    # Create a new RGB image with the desired dimensions.
    img = Image.new("RGB", (desired_width, desired_width))
    img.frombytes(output_rgb.tobytes(), "raw", "RGBX", 0, -1)
    img.show()

In [8]:
false_grayscale(1000, 300)

Lenght of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 4.382652282714844 seconds


**Pure Grayscale Plot**

In a pure grayscale image, every pixel is represented by a single intensity value from 0 (black) to 255 (white). The number of iterations required for each point in the Julia set calculation directly maps to a grayscale intensity. Points that take more iterations to reach the escape condition are assigned a higher value (lighter), and those that escape quickly are assigned a lower value (darker).

In [9]:
def pure_grayscale(desired_width, max_iterations):
    output = calc_pure_python(desired_width, max_iterations, return_output=True)
    
    # Maximum iteration count as the scale factor for normalization.
    scale_factor = float(max(output))
    
    # Normalize the iteration counts to the 0-255 grayscale range.
    # Values are scaled linearly based on the maximum number of iterations.
    # The result is a pure grayscale value, meaning each pixel's color intensity
    # directly reflects the number of iterations.
    scaled = [int(o / scale_factor * 255) for o in output]
    
    # Convert the scaled iteration counts into an array of unsigned bytes ('B').
    # Each byte represents a pixel's grayscale color value in the image.
    output = array.array('B', scaled)
    
    # Create a new grayscale image.
    img = Image.new("L", (desired_width, desired_width))
    img.frombytes(output.tobytes(), "raw", "L", 0, -1)
    img.show()

In [10]:
pure_grayscale(desired_width=1000, max_iterations=300)   

Lenght of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 4.300005197525024 seconds


## Step 2

Defining a new function `timefn` which takes a function as an argument: the inner function, `measure_time`,  takes `*args` (a variable number of positional arguments) and `*kwargs` (a variable number of key/value arguments) and passes them through to `fn` for exection.

In [11]:
from functools import wraps

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        start = time.time()
        result = fn(*args, **kwargs)
        end = time.time()
        print("@timefn:" + fn.__name__ + " took " + str(end - start) + " seconds")
        return result
    return measure_time

Decorating `calculate_z_serial_purepython` with `@timefn`to profile it.

In [12]:
@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    '''Calculate output list using Julia update rule'''
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

Testing the new version:

In [13]:
if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python(desired_width=1000, max_iterations=300)

Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.299396514892578 seconds
calculate_z_serial_purepython took 4.299456834793091 seconds


## Step 3

Using the `timeit` module to get a coarse measurement of the execution speed of the CPU-bound function. Running 10 loops with 5 repetitions. First executing the measurement on the command line with `python -m timeit -r 5 -n 10 -s "from benchmarking_and_profiling import calc_pure_python" "calc_pure_python(desired_width=1000, max_iterations=300)"` looks like this:

<img src="timeit.png">

Now, making the measurement in this notebook:

In [14]:
%timeit -r 5 -n 10 calc_pure_python(desired_width=1000, max_iterations=300)

Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.134695291519165 seconds
calculate_z_serial_purepython took 4.134755373001099 seconds
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.087386608123779 seconds
calculate_z_serial_purepython took 4.087445259094238 seconds
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.086347818374634 seconds
calculate_z_serial_purepython took 4.0863938331604 seconds
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.340664625167847 seconds
calculate_z_serial_purepython took 4.340715646743774 seconds
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.124864101409912 seconds
calculate_z_serial_purepython took 4.124917507171631 seconds
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.111462116241455 seconds
calculate_z_serial_purepython t

Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.262952089309692 seconds
calculate_z_serial_purepython took 4.2630088329315186 seconds
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.2769012451171875 seconds
calculate_z_serial_purepython took 4.276954889297485 seconds
4.43 s ± 58 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


## Step 4

Using the `cProfile` module to profile the code. I will sort the results by the time spent inside each function. This will give a view into the slowest parts. First, showing the results in command line using:

`$python -m cProfile -s cumulative benchmarking_and_profiling.py`

<img src="cProfile.png">

The profiling indicates a total of 36257870 calls in 11.61 seconds. By the image, we can see that the entry point to the code takes a total of 11 seconds. Inside `calc_pure_python`, the call `calculate_z_serial_purepython` consumes 10.90 seconds, and both functions are only called once.

Inside `calculate_z_serial_purepython`, the time spent on lines of code is 8.21 seconds. This function makes 34219980 calls to abs, which take a total of 2.69 seconds, along with some other calls that do not cost much time.

Now, making the measurement in this notebook:

In [15]:
import cProfile
import pstats

# Creating a Profile object
profiler = cProfile.Profile()

# Profiling the calc_pure_python function

profiler.enable()  # Start collecting profiling data
calc_pure_python(desired_width=1000, max_iterations=300)
profiler.disable() # Stop collecting profiling data

# Create a Stats object from the data and sort it by cumulative
stats = pstats.Stats(profiler).sort_stats('cumulative')
stats.print_stats()

Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 10.472025394439697 seconds
calculate_z_serial_purepython took 10.472108125686646 seconds
         36222178 function calls in 11.120 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   11.141    5.570 /home/juli/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3490(run_code)
        2    0.000    0.000   11.141    5.570 {built-in method builtins.exec}
        1    0.537    0.537   11.120   11.120 /tmp/ipykernel_80318/486211223.py:4(calc_pure_python)
        1    0.000    0.000   10.472   10.472 /tmp/ipykernel_80318/671623914.py:4(measure_time)
        1    7.866    7.866   10.472   10.472 /tmp/ipykernel_80318/2689343387.py:1(calculate_z_serial_purepython)
 34219980    2.606    0.000    2.606    0.000 {built-in method builtins.abs}
  2002000    0.105    0.000    0.105    0.000 {method 'append

In this case, the results are just a little bit different. At the start, the `run_code` method is the one used to execute code in the Jupyter environment and is called 2 times because of the execution of the code blocks in the notebook cells.

The rest, is somewhat similar.

The following is the image I'm making reference from my computer:

<img src="cProfileJupyter.png">

## Step 5

Using `snakeviz` to get a high-level understanding of the cProfile statistics file. I will analyze the output and make a syntesis of the findings.

To run `snakeviz` correctly I used the following commands:

`$ python -m cProfile -o benchmarking_and_profiling.cprof benchmarking_and_profiling.py`              
 
`$ snakeviz benchmarking_and_profiling.cprof`

And this was the result:

<img src="snakeviz.png">

In this image, each block represents a function call, and the size of the block is proportional to the time spent in that function, including the cumulative time.

The root block represents the total time taken by the script, which is around 11.5 seconds. The `run_code` function within IPython's core `interactiveshell.py` module is at the top of the call stack, indicating that the profiled code was run within an IPython environment. Below, there is the `<module>` function from `benchmarking_and_profiling.py` with a cumulative time of 11.55 seconds. This indicates that nearly all the time taken for the script execution is within this module. Within the `benchmarking_and_profiling.py` module, the calc_pure_python function was called and has a cumulative time of 11.2 seconds, indicating that it's the main driver of the computation time.

The function `calculate_z_serial_purepython` consumes most of the execution time, with a total time of 10.6 seconds. This suggests that the function is highly CPU-intensive since is where the core computation logic for generating the Julia set is located.

Also, it can be seen on the lilac block of the end the `abs` function, that took a total of 2.7 seconds.

Now, let's use `SnakeViz` in Jupyter notebook to compare results with the results from my computer:

In [16]:
%pip install snakeviz
%load_ext snakeviz
%snakeviz calc_pure_python(desired_width=1000, max_iterations=300)

Note: you may need to restart the kernel to use updated packages.
Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 10.355187892913818 seconds
calculate_z_serial_purepython took 10.35526418685913 seconds
 
*** Profile stats marshalled to file '/tmp/tmptuprqvu9'.
Embedding SnakeViz in this document...
<function display at 0x7efcf474b100>


## Step 6

Using the `line_profiler` and `kernprof` file to profile line-by-line the function `calculate_z_serial_purepython`.

To run it correctly I used:

`$ kernprof -l -v benchmarking_and_profiling.py`

<img src='kernprof.png'>

Analyzing this results, it can be seen that this function had a total run time of 26.08 seconds. Line 76, which contains the while loop was hitted 34219980 times, which consumes 46% of the total time within the function, this makes sense because is where the core computation for the set occurs. The timeper hit is small for all lines, but because the function is called many times, the cumulative time then becomes significant.

From this, we can conclude that teh function is called many times, which makes that even operations that normally take small amount of time per call can add up to a significant portion of the total run time.

Now, run the `line_profiler` and `kernprof` in this notebook to compare the results:

In [17]:
%pip install line_profiler
%load_ext line_profiler
%lprun -f calculate_z_serial_purepython calc_pure_python(desired_width=1000, max_iterations=300)

Note: you may need to restart the kernel to use updated packages.


/home/juli/anaconda3/lib/python3.11/site-packages/line_profiler/ipython_extension.py:97: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding calculate_z_serial_purepython.__wrapped__ instead.
  profile = LineProfiler(*funcs)


Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 14.04855489730835 seconds
calculate_z_serial_purepython took 14.04864501953125 seconds


## Step 7

Using `memory_profiler` to diagnose memory usage.

To run it correctly I used:

`$ python -m memory_profiler benchmarking_and_profiling.py`

And this are my results:
<img src="memory_profiler1.png">

**calc_pure_python**

In the image it can be seen that the memory usage starts at 25.31 MiB before the function begins. As the function sets up initial variables and enters loops to populate the lists, there is no significant change in memory usage. This suggests that these lists are relatively small. A substantial memory increment occurs as the lists `zs` and `cs` are populated inside the nested loops. The memory jumps by 287.15 MiB to a total of 101.82 when creating these lists, which suggests that these lists contain a large number of complex numbers, but the largest memory increment occurs when appending `zs` and `cs` because these lists are very large.

The final recorded memory usage after the function is completed, as will be seen in the next image, is 113.10 MiB, which is higher than the initial usage.

<img src="memory_profiler2.png">

**calculate_z_serial_purepython**

In this case, the initial memory of the function is 101.82 MiB. The function increases memory usage by 7.65 MiB when it creates the list `output`, this being the only significant memory increment within the function.

Now, run `memory_profiler` to compare results:

In [18]:
!pip install memory_profiler
%load_ext memory_profiler
%memit calc_pure_python(desired_width=1000, max_iterations=300)

Lenght of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 4.152618646621704 seconds
calculate_z_serial_purepython took 4.152669429779053 seconds
peak memory: 175.87 MiB, increment: 62.36 MiB


## Conclusion

This exercise was useful to understand benchmarking and profiling, essential practices that can help programmers to make better and optimized functions and code with efficient resource management and that can also enhance user experience. Using this techniques, bottlenecks and inefficient resource use can be identified. Ultimately, regular use of the techniques explored can very much improve and deliver high-quality programs.

_All the code is in a .py file too!_